## 导入工具包

In [1]:
from tools25 import * 

## 1, 登录
在tools25文件的login方法中填写用户名和密码后保存然后来到本文件Restart Kernal后重新import tools23

In [2]:
s = login()

b'{"user":{"id":"PS30320"},"token":{"expiry":14400.0},"permissions":["TUTORIAL"]}'


## 2, 根据dataset_id，调取平台API获取datafields

In [3]:
df = get_datafields(s, dataset_id = 'option8', region='USA', universe='TOP3000', delay=1)
print(df)

                           id  \
0    historical_volatility_10   
1   historical_volatility_120   
2   historical_volatility_150   
3   historical_volatility_180   
4    historical_volatility_20   
..                        ...   
59   parkinson_volatility_180   
60    parkinson_volatility_20   
61    parkinson_volatility_30   
62    parkinson_volatility_60   
63    parkinson_volatility_90   

                                          description  \
0   Close-to-close Historical volatility over 10 days   
1   Close-to-close Historical volatility over 120 ...   
2   Close-to-close Historical volatility over 150 ...   
3   Close-to-close Historical volatility over 180 ...   
4   Close-to-close Historical volatility over 20 days   
..                                                ...   
59  Parkinson model's historical volatility over 1...   
60  Parkinson model's historical volatility over 2...   
61  Parkinson model's historical volatility over 3...   
62  Parkinson model's historical 

### Matrix Data

In [4]:
# print(df[df['type'] == "MATRIX"]["id"].tolist())
pc_fields = process_datafields_opt(df, "matrix")
print(pc_fields[:3])
print(len(pc_fields))

['winsorize(ts_backfill(implied_volatility_call_10/implied_volatility_mean_10, 120), std=4)', 'winsorize(ts_backfill(implied_volatility_call_10/implied_volatility_mean_1080, 120), std=4)', 'winsorize(ts_backfill(implied_volatility_call_10/implied_volatility_mean_120, 120), std=4)']
288


## 3, Alpha factory（一阶Alpha模板，生成Matrix数据之后，开始跑这个)
### start with First Order

In [5]:
ops_lst=get_ops(s)['name'].to_list()    # 获取你账号可用的ops
ops=intersection_of_lists(ops_lst,ts_ops)
first_order = get_first_order(pc_fields, ops,prefix='') # 新增prefix,提效first order
print(first_order[:10])
print(len(first_order))

['ts_mean', 'ts_sum', 'ts_std_dev', 'ts_scale', 'ts_rank', 'ts_quantile', 'ts_zscore', 'ts_arg_min', 'ts_product', 'ts_arg_max', 'ts_delta']
['winsorize(ts_backfill(implied_volatility_call_10/implied_volatility_mean_10, 120), std=4)', 'ts_mean(winsorize(ts_backfill(implied_volatility_call_10/implied_volatility_mean_10, 120), std=4), 5)', 'ts_mean(winsorize(ts_backfill(implied_volatility_call_10/implied_volatility_mean_10, 120), std=4), 22)', 'ts_mean(winsorize(ts_backfill(implied_volatility_call_10/implied_volatility_mean_10, 120), std=4), 66)', 'ts_mean(winsorize(ts_backfill(implied_volatility_call_10/implied_volatility_mean_10, 120), std=4), 120)', 'ts_mean(winsorize(ts_backfill(implied_volatility_call_10/implied_volatility_mean_10, 120), std=4), 240)', 'ts_sum(winsorize(ts_backfill(implied_volatility_call_10/implied_volatility_mean_10, 120), std=4), 5)', 'ts_sum(winsorize(ts_backfill(implied_volatility_call_10/implied_volatility_mean_10, 120), std=4), 22)', 'ts_sum(winsorize(ts_back

In [6]:
fo_alpha_dict = defaultdict(list)
init_decay = 5
regions = ["usa"]
for key in regions:
    for alpha in first_order:
        fo_alpha_dict[key].append((alpha,init_decay))

## 4, simulate alphas

In [ ]:
# Simulate First Order
region_dict = {"usa" : ("USA", "TOP3000")}

first_layer_bag = []
simulate(fo_alpha_dict, region_dict, "opt_fo_usa", 'SUBINDUSTRY', 0 , first_layer_bag)

b'{"user":{"id":"PS30320"},"token":{"expiry":14400.0},"permissions":["TUTORIAL"]}'
curr usa len 16128
group 0 winsorize(ts_backfill(implied_volatility_call_10/implied_volatility_mean_10, 120), std=4) USA TOP3000 5
201
group 0 ts_mean(winsorize(ts_backfill(implied_volatility_call_10/implied_volatility_mean_10, 120), std=4), 5) USA TOP3000 5
201
group 0 ts_mean(winsorize(ts_backfill(implied_volatility_call_10/implied_volatility_mean_10, 120), std=4), 22) USA TOP3000 5
201
group 0 post done
2025-03-20 22:05:19.631967------------------
https://api.worldquantbrain.com/simulations/wXdQS4Jz5199lr1hG2yKGjD done simulating, getting alpha details
https://api.worldquantbrain.com/simulations/3mG4JK8AN5dKbsCHIXOhzEb done simulating, getting alpha details
https://api.worldquantbrain.com/simulations/2Ol33efi952SbyU162uotdtw done simulating, getting alpha details
group 0 USA simulate done
group 1 ts_mean(winsorize(ts_backfill(implied_volatility_call_10/implied_volatility_mean_10, 120), std=4), 66) USA

## 5, Select alphas(根据时间取出刚刚simulate的alpha，用于跑二阶模板)
go to web alphas penal to look for the number and date to track for next order improve

In [ ]:
## get promising alphas to improve in the next order
# 注意这里的时间是纽约时间，比北京时间慢13个小时
from tools25 import *


fo_tracker = get_alphas("01-11", "01-12", 1.0, 0.5, "USA", 320, "track")

## 6, Next order improvement - Second Order(二阶模板，对一阶Alpha中性化处理)
second order: ts_ops(field, days) -> group_ops(ts_ops(field, days), group)

In [ ]:
s=login()
print(len(fo_tracker['next']))
print(len(fo_tracker['decay']))

fo_layer = prune(fo_tracker['next'] + fo_tracker['decay'], 'usa', 'fnd6', 5)

ops_lst=get_ops(s)['name'].to_list()    # 获取你账号可用的ops
group_ops = ["group_neutralize", "group_rank", "group_normalize", "group_scale", "group_zscore"]
ops=intersection_of_lists(ops_lst,group_ops)
flst=get_fields_all()       # 获取你的可用字段

so_alpha_dict = defaultdict(list)
for key, couples in fo_layer.items():
    for expr, decay in couples:
        for alpha in get_group_second_order_factory([expr], ops, key,flst):
            so_alpha_dict[key].append((alpha,decay))


# print(so_alpha_dict['usa'][:3])
print(len(so_alpha_dict['usa']))


### Simulate second order

In [ ]:
region_dict = {"usa" : ("USA", "TOP3000")}

so_bag = []
simulate(so_alpha_dict, region_dict, "opt_so_usa", 'SUBINDUSTRY', 0, so_bag)

## Get submittable alphas（检查simulatied alphass是否可以submit）
check_submission and view alphas

In [ ]:
# 输入你simulate运行日期，这里的时间都是纽约时间，比北京时间慢13小时
# 如果你是24号上午运行的simulate，那么这里的时间应该是23-24
import traceback


th_tracker = get_alphas("01-09", "01-10", 1.25, 1, "USA", 320, "submit")
stone_bag = []
for alpha in th_tracker['next'] + th_tracker['decay']:
    stone_bag.append(alpha[0])  # 取出其中的alpha_id
stone_bag_len=len(stone_bag)
print(fr"stone_bag len {stone_bag_len}")
gold_bag_all = []

for s_step in range(0, stone_bag_len, 10):
    e_step=s_step+10
    if e_step>=stone_bag_len:
        e_step=stone_bag_len-1

    print(fr"stone_bag s_step:{s_step},e_step:{e_step}")
    stone_bag_temp=stone_bag[s_step:e_step]


    gold_bag = []
    try:
        check_submission(stone_bag_temp, gold_bag, 0)
    except Exception as e:
        traceback.print_exc()
        print(fr"check_submission unknown error")   
        print(fr"time now:{get_timenow()}") 

    print("___________________________________________________________________________________________________")
    print(fr"time now:{get_timenow()}")
    gold_bag_all+=gold_bag
    print(gold_bag)

In [ ]:
# 可以尝试submit的alpha_id
# https://platform.worldquantbrain.com/alpha/6lmK2XO
# 替换掉以上链接末尾的6lmK2XO就可以在网页打开
gold_bag_all